### Contents

1. Investigate pypi packages
2. Jmespath
3. Pyparsing



# Training Day summary

I went looking for trouble on PyPi - https://hugovk.github.io/top-pypi-packages/

Of the top 30 (Based on downloads over the last 30 days)

| name | status |
| --- | --- |
|boto3|AWS Stuff|
|botocore | AWS Stuff|
|urllib3 | Using |
|setuptools | Using |
| requests| Using |
|s3transfer |AWS Stuff|
|google-api-core |Google Stuff|
|charset-normalizer|Read text from an unknown encoding|
|certifi|Mozilla certificate stuff|
|python-dateutil| Interesting, not used|
|idna|Iñternatiønalized Dömain Naméş|
|typing-extensions|Backporting typing features|
|six|Used in the past, hopefully gone|
|pyyaml|Aware of|
|cryptography|Aware of|
|awscli|AWS Stuff|
|numpy |Used|
|Wheel | Using|
|**pyparsing**| **Interesting, investigate** |
|rsa|aware of|
|packaging| aware of|
|pyasn1|[Abstract Syntax Notation - An ISO Standard](https://en.wikipedia.org/wiki/ASN.1) |
|pip|Using a lot|
|pytz|extra time zone information|
|**jmespath** | **Interesting, investigate** |
|pyjwt|JSON Web Token (?)|
|importlib-metadata| Get Python Metadata (commercial license)|
|pandas|Used|
|zipp|Support for zipfiles /prototype backport for pathlib.zipfile|
|protobuf|Aware of|


## jmspath (James-Path)




In [50]:
%%writefile example.json
{
	"foo": {
		"bar": 400,
		"baz": "Rabbits",
		"qux": ["widely", "regarded", "as", "a", "bad", "move"]
	},
    "bar": [
        {"x": 42, "y": 23},
        {"x": 43, "y": 45}
    ]
}

Overwriting example.json


In [51]:
import jmespath as jp
import json
from pathlib import Path

data = json.loads(Path('example.json').read_text())

data

{'foo': {'bar': 400,
  'baz': 'Rabbits',
  'qux': ['widely', 'regarded', 'as', 'a', 'bad', 'move']},
 'bar': [{'x': 42, 'y': 23}, {'x': 43, 'y': 45}]}

a direct jamespath search

In [21]:
jp.search('foo.qux', data)

['widely', 'regarded', 'as', 'a', 'bad', 'move']

a compiled jamespath search

In [23]:
my_jp = jp.compile('foo.baz')
my_jp.search(data)

'Rabbits'

Simple wildcard searching

In [28]:
jp.search('foo.*', data)

[400, 'Rabbits', ['widely', 'regarded', 'as', 'a', 'bad', 'move']]

Comparisons

In [52]:
jp.search('bar[?x>y]', data)

[{'x': 42, 'y': 23}]

You can get functions too

In [54]:
jp.search("length(foo)", data)

3

In [65]:
jp.search("join(`, `, foo.qux)", data)

'widely, regarded, as, a, bad, move'

In [66]:
jp.search('keys(@)', data)

['foo', 'bar']

### JmesPath Conclusion 

Essentially a mini language with far more scope than I have time or energy to learn. It has, however, some useful functionality, which is worth being aware of.


## Pyparsing

I tried to write a mini parser for the `rose-eg.conf` format.



In [9]:
from pathlib import Path
from pyparsing import *

BASIC = """
FOO=bar
"""

TESTSTR = """
# This is a comment
foo="bar"
[cow]
birth="calving"
baby="calf"

# legs=4
# stomachs=4

[alpaca]
# legs=4
birth="unpacking"
baby="cria"
# stomachs=3

[squid]
"""

EQ, LSQ, RSQ = map(Suppress, "=[]")
END = StringEnd()
COMMENT = Regex(r'#.*')

def evalValue(tokens):
    import ast
    return [ast.literal_eval(tokens[0])]


key = Word(alphas)("key")
val = Word(alphas)("value")
val = val.setParseAction(evalValue)

kvpair = key + EQ + val
section = LSQ + Word(alphanums)('section') + RSQ

# overall = Group(
#     ZeroOrMore(
#         Group(kvpair)
#     )
#     + ZeroOrMore((
#         section + ZeroOrMore(Group(kvpair))
#     ))
# )
overall = kvpair
overall.ignore(COMMENT)

tree = overall.parseString(BASIC)

tree

ValueError: malformed node or string on line 1: <ast.Name object at 0x7f0765116440>

## Pyparsing conclusion

It may well be better than writing ones own parsing library from scratch, if you haven't already done that. The concept seems very pythonic, but overall it feels complicated and clunky. I'm not at all sure I'd use this, I think I prefer regexes.